In [176]:
!pip install dash

In [177]:
pip install google-search-results

In [178]:
pip install requests

In [179]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote
import urllib, requests, socket, re, lxml, io, bs4, sqlite3, pandas, sqlalchemy
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import date
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import dash
from dash import dcc, html
import plotly.graph_objects as go


In [180]:
pd.set_option('display.max_rows', 500)

In [181]:
token = 'b5af8e86b5af8e86b5af8e86a5b682c52abb5afb5af8e86d273a8a7fb8da536cfc71a33'
group_shaurma_ids = [ '224707598', '205996417','216834249', '223718016', '179827898', '221949082', '193882078', '221312434']
version = '5.199'

def change_number_to_real_SEX(x):
  if x == 2:
    return 'man'
  if x == 1:
    return 'woman'
  else:
    return 'unknown'

def calculate_age(x):
    born = date(int(x[0]), int(x[1]), int(x[2]))
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))


def change_date_of_birth_to_normal(x):
  return x.split('.')[::-1]

def group_members(group_id, max_count = 5000):

    offset = 0
    count = 1000
    all_members = []

    while offset<=max_count:
      url = 'https://api.vk.com/method/groups.getMembers'
      params = {
          'group_id': group_id,
          'count': count,
          'offset': offset,
          'fields': 'id,first_name,last_name, bdate, sex, education',
          'access_token': token,
          'v': version
      }

      response = requests.get(url, params=params)
      data = response.json()

      members = data.get('response', {}).get('items', [])

      if not members:
          break

      all_members.extend(members)
      offset += 1000
    return all_members

all_shaurma_members = pd.DataFrame()

for id in group_shaurma_ids:
  members_df = pd.DataFrame(group_members(id))


  members_df.drop(columns = ['deactivated', 'can_access_closed', 'is_closed',  'faculty', 'faculty_name', 'education_form', 'education_status'], inplace = True, errors="ignore")
  members_df.dropna(inplace = True)

  members_df['sex'] = members_df['sex'].apply(change_number_to_real_SEX)
  members_df = members_df[members_df['sex'].isin(['man', 'woman'])]


  members_df['bdate'] =members_df['bdate'].apply(change_date_of_birth_to_normal)
  members_df = members_df[members_df['bdate'].apply(len) == 3]
  members_df['age'] = members_df['bdate'].apply(calculate_age)
  members_df.drop(columns = [ 'bdate'], inplace = True)

  members_df = members_df[members_df['university_name'].apply(len) > 0]

  all_shaurma_members = pd.concat([all_shaurma_members, members_df], ignore_index=True)

all_shaurma_members.loc[all_shaurma_members['age'] > 80, 'age'] = all_shaurma_members['age'].mean()
all_shaurma_members['university_name'] = all_shaurma_members['university_name'].str.replace('\r\n', '', regex=False)



<ipython-input-181-f7a751a8c950>:73: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '36.23916292974589' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [182]:
woman_shurma_members = all_shaurma_members.loc[all_shaurma_members['sex']=='woman']
man_shurma_members = all_shaurma_members.loc[all_shaurma_members['sex']=='man']


In [ ]:
sex_counts = all_shaurma_members['sex'].value_counts().reset_index()
sex_counts.columns = ['sex', 'count']

figs = {}

fig = px.pie(sex_counts,
             values='count',
             names='sex',
             hole=0.4,
             color_discrete_sequence=px.colors.qualitative.Pastel,
             labels={
                 'count': 'Количество',
                 'sex': 'Пол'
             },
             title='Распределение по полу',
             width=800,
             height=600
            )


fig.show()

figs['fig1'] = fig

In [ ]:

data = {"Все посетители": all_shaurma_members,"Женщины": woman_shurma_members,"Мужчины": man_shurma_members}

colors = ['#636EFA', '#EF553B', '#00CC96']

count_ = 2
for people in data:
  fig = px.histogram(data[people], x='age', nbins=20, title=f'Распределение возраста: {[people]}', color_discrete_sequence=[colors[list(data).index(people)]])

  fig.update_layout(xaxis_title='Возраст',yaxis_title='Количество людей',bargap=0.1)
  figs[f'fig{count_}'] = fig
  count_ += 1
  fig.show()


In [ ]:
top_universities = all_shaurma_members['university_name'].value_counts().nlargest(10)

fig = px.bar(x=top_universities.index,y=top_universities.values,labels={'x': 'Университет', 'y': 'Количество студентов'},title='Топ-10 университетов по количеству студентов')

fig.show()

In [ ]:
from serpapi import GoogleSearch

def get_second_key_value(dict_, number_of_dict):


    second = list(dict_.keys())[number_of_dict]

    return dict_[second]

start_date = '2016-01-01'
end_date = '2025-01-01'

params = {
    "engine": "google_trends",
    "q": "шаурма,бургер,перепечи,чебуреки,чурос",
    "data_type": "TIMESERIES",
    "api_key": "f0ab22ef2df91c0f329933cd799dd94c96cb7b9f27bca27b8df2ff2c1765d09d",
    "date": "today 5-y"
}

search = GoogleSearch(params)
results = search.get_dict()

needed_data = pd.DataFrame(results["interest_over_time"]['timeline_data'])

popularity_df = needed_data.drop(columns = ['timestamp', 'partial_data'])

bluda = ['шаурма', 'бургер','перепечи','чебуреки','чурос']

for i in range(5):
    newcolumn = bluda[i]
    popularity_df[newcolumn] = popularity_df['values'].apply(lambda x: get_second_key_value(x[i], 2))
popularity_df=popularity_df.drop(columns = ['values'])

In [ ]:
def parse_date(date_str):
    try:
        start_date, year = date_str.split(', ')
        start_date = start_date.split('–')[0].strip()  # Извлекаем начальную дату
        return pd.to_datetime(f"{start_date}, {year}", format='%b %d, %Y')
    except:
        return None  # Если дата некорректна, возвращаем None.  ###ЭТО НАПИСАЛ ГПТ


popularity_df['date'] = popularity_df['date'].apply(parse_date)


popularity_df = popularity_df.dropna(subset=['date'])

fig = px.line(popularity_df, x='date', y=['шаурма', 'бургер', 'перепечи', 'чебуреки', 'чурос'],
              title='Популярность блюд по времени',
              labels={'value': 'Популярность', 'date': 'Дата', 'variable': 'Блюдо'})

fig.show()



In [ ]:
figs['fig5'] = fig


In [183]:
c = 0
def user_subscriptions(user_id, extended=1, max_count=60):

    offset = 0
    count = 20
    all_subs = []

    while offset < max_count:
      url = 'https://api.vk.com/method/users.getSubscriptions'
      params = {
          'user_id': user_id,
          'extended': extended,
          'access_token': token,
          'v': version,
          'fields': 'name',
          'count': count
      }

      response = requests.get(url, params=params)
      data = response.json()

      subs = data.get('response', {}).get('items', [])
      if not subs:
        break

      all_subs.extend(subs)
      offset += count

    return all_subs

all_subs = pd.DataFrame()
for id in all_shaurma_members['id']:

    subs = user_subscriptions(id)
    if subs:
      subs_df = pd.DataFrame(subs)
      if 'name' in subs_df.columns:
        all_subs = pd.concat([all_subs, subs_df['name']], ignore_index=True)
        c+=1
        print(c)


all_subs =all_subs.dropna()

all_subs_str=[]
for group_name in all_subs[0]:
  group_name = group_name.replace(' ', '').lower()
  all_subs_str.append(group_name)

print(all_subs)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [190]:
fig_sport = px.pie(dataframes_dict["спорт"], names="спорт", values="count", title="Распределение по видам спорта", width=400, height=400)
fig_countries = px.pie(dataframes_dict["страны"], names="страны", values="count", title="Распределение по странам", width=400, height=400)
fig_music = px.pie(dataframes_dict["музыка"], names="музыка", values="count", title="Распределение по жанрам музыки",width=400, height=400)


fig_sport.show()
fig_countries.show()
fig_music.show()

figs['fig6'] = fig_sport
figs['fig7']= fig_countries
figs['fig8'] =fig_music

In [187]:
interests = [
    ["футбол", "баскетбол", "волейбол", "теннис", "бокс", "мма", "формула 1", "хоккей"],  # спорт
    ["фильм", "сериал", "аниме", "документалка", "мультфильм", "боевик", "драма", "комедия"],  # кино
    ["финансы", "крипта", "акции", "инвестици", "банк"],  # финансы
    ["готовк", "ресторан", "фастфуд", "зож", "кулинарное шоу", "десерт", "напитк"],  # еда
    ["рэп", "поп", "рок", "металл", "классика", "электро", "джаз"],  # музыка
    ["компьютерные", "консольные", "мобильные", "киберспорт", "vr"],  # игры
    ["франция", "германия", "испания", "аргентина", "италия", "китай"],  # страны
    ["авто", "мото", "электромобил", "гонки", "тест-драйв", "обзоры авто", "ремонт машин"],  # авто
    ["здоровье", "фитнес", "йога", "медитация", "диет", "психика", "спортзал"],  # здоровье
]

categories = ["спорт", "кино", "финансы", "еда", "музыка", "игры", "страны", "авто", "здоровье"]


dataframes_dict = {}

for i in range(len(categories)):
  df = pd.DataFrame({categories[i]: interests[i],'count': 0})

  dataframes_dict[categories[i]] = df

all_subs_np = np.array(all_subs_str)

for category in categories:
  for i in dataframes_dict[category][category]:
    count = np.sum(np.char.count(all_subs_np, i))
    dataframes_dict[category].loc[dataframes_dict[category].index[dataframes_dict[category][category] == i].tolist(), 'count']  +=count
dataframes_dict

for category in categories:
  print(dataframes_dict[category].sort_values(by='count',ascending=False))

       спорт  count
0     футбол    156
4       бокс     90
5        мма     51
7     хоккей     30
1  баскетбол     21
2   волейбол      6
3     теннис      3
6  формула 1      0
           кино  count
0         фильм    732
1        сериал    327
2         аниме     81
4    мультфильм     45
5        боевик      6
6         драма      3
3  документалка      0
7       комедия      0
     финансы  count
4       банк     54
2      акции     21
3  инвестици     12
0    финансы      6
1     крипта      0
              еда  count
3             зож     93
5          десерт     54
1        ресторан     51
0          готовк     39
6          напитк      9
2         фастфуд      0
4  кулинарное шоу      0
     музыка  count
2       рок    480
1       поп    177
5   электро     60
3    металл     24
0       рэп     21
6      джаз     12
4  классика      6
           игры  count
4            vr     51
0  компьютерные     21
3    киберспорт      9
2     мобильные      0
1    консольные      0
   

In [193]:


app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Мой дашборд", style={'textAlign': 'center'}),

    html.Div([
        dcc.Graph(
            figure=figs['fig1'],
            style={'width': '100%', 'height': '400px'}
        ),
        dcc.Graph(
            figure=figs['fig2'],
            style={'width': '100%', 'height': '400px'}
        ),
        dcc.Graph(
            figure=figs['fig3'],
            style={'width': '100%', 'height': '400px'}
        ),
        dcc.Graph(
            figure=figs['fig4'],
            style={'width': '100%', 'height': '400px'}
        ),
        dcc.Graph(
            figure=figs['fig5'],
            style={'width': '100%', 'height': '400px'}
        ),
    ], style={'display': 'grid', 'gridTemplateColumns': 'repeat(3, 1fr)', 'gap': '10px'}),
])


if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>